# CAO Points

## Introduction

The CAO points data, available from the [CAO website](http://www.cao.ie), is published in a different format for each of the years 2019, 2020, and 2021. Each year's data, therefore, requires a different approach to acquisition, conversion to pandas dataframe, and cleaning. The 2019 data is published in two pdf files; one for level 8 courses, and one for levels 6 and 7. The 2020 data is published as an Excel spreadsheet, and the 2021 data as preformatted text in a HTML webpage.

The sttributes of interest for comparison between the various years' datasets are `Course Code`, `Course Name`, `Institution Name`, `EOS`, which is the number of points achieved by the last applicant to be offered a place on the course, and `Mid`, which is the mid point between the number of points held by the highest point score and the lowest point score of the applicants offered a place on the course [1]. The 2021 data does not explicitly contain an either an `EOS` or a `Mid` column. It does provide the *Round 1* and *Round 2* points required for entry into each course as `RND1` and `RND2`. Examination of the 2020 data, which contains both an `EOS` field *and* `RND1` and `RND2` fields demonstrates that the `EOS` field is equal to the `RND2` value if it exists, otherwise the `RND1` value (```EOS = RND1 if RND1 else RND2```). As for the `Mid` field; this information does not appear to be available yet for the 2021 data.

In [1]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
from tabula import read_pdf
import requests
from requests.compat import urljoin
import os


## Acquiring the data

In [2]:
# Location of CAO points data
base_url = 'http://www2.cao.ie/points/'
# Local data directory
data_dir = 'data/cao'

# Dictionary of source file names mapped to the file names that will be used locally
points_urls  = ({'lvl8_19.pdf': 'lvl8_19.pdf', 
                 'lvl76_19.pdf': 'lvl76_19.pdf', 
                 'CAOPointsCharts2020.xlsx': 'all2020.xlsx', 
                 'l8.php': 'lvl8_21.html',
                 'l76.php': 'lvl76_21.html'})

# for each of the source files construct url, 
# fetch file, and write to data directory
for url in (points_urls):
    response = requests.get(urljoin(base_url, url))
    with open(os.path.join(data_dir, points_urls[url]), 'wb') as f:
        f.write(response.content)

### 2019 Points Data

The 2019 points data is held in two PDF files, one for level 8 courses and one for levels 6 and 7.

#### Level 8

In [3]:
# read the entire pdf, extracting tables into a single dataframe
df8 = read_pdf(os.path.join(data_dir, "lvl8_19.pdf"), pages="all", multiple_tables=False)[0]
df8.head(10)

,Course Code,INSTITUTION and COURSE,EOS,Mid
0,NaN,Athlone Institute of Technology,NaN,NaN
1,AL801,Software Design with Virtual Reality and Gaming,304,328
2,AL802,Software Design with Cloud Computing,301,306
3,AL803,Software Design with Mobile Apps and Connected...,309,337
4,AL805,Network Management and Cloud Infrastructure,329,442
5,AL810,Quantity Surveying,307,349
6,AL820,Mechanical and Polymer Engineering,300,358
7,AL830,General Nursing,410,429
8,AL832,Psychiatric Nursing,387,403
9,AL836,Nutrition and Health Science,352,383


In [4]:
# Create a new column in the dataframe for institution name 
# identify institution name rows as those containing null course codes
# and add those institution names to the new institution column
df8['Institution'] = df8[df8['Course Code'].isnull()]['INSTITUTION and COURSE']
df8.rename(columns={'INSTITUTION and COURSE':'Course Name'}, inplace=True)
df8.head()

,Course Code,Course Name,EOS,Mid,Institution
0,NaN,Athlone Institute of Technology,NaN,NaN,Athlone Institute of Technology
1,AL801,Software Design with Virtual Reality and Gaming,304,328,NaN
2,AL802,Software Design with Cloud Computing,301,306,NaN
3,AL803,Software Design with Mobile Apps and Connected...,309,337,NaN
4,AL805,Network Management and Cloud Infrastructure,329,442,NaN


In [5]:
# Fill empty fields in the institution column with the most recent non-na field
df8['Institution'] = df8['Institution'].fillna(method='ffill')
df8.head()

,Course Code,Course Name,EOS,Mid,Institution
0,NaN,Athlone Institute of Technology,NaN,NaN,Athlone Institute of Technology
1,AL801,Software Design with Virtual Reality and Gaming,304,328,Athlone Institute of Technology
2,AL802,Software Design with Cloud Computing,301,306,Athlone Institute of Technology
3,AL803,Software Design with Mobile Apps and Connected...,309,337,Athlone Institute of Technology
4,AL805,Network Management and Cloud Infrastructure,329,442,Athlone Institute of Technology


In [6]:
# Finally, remove rows containing only institution names
df8 = df8[df8['Course Code'].notna()]

# Set some display options
# pd.set_option("display.max_rows", None)
# pd.set_option("display.max_colwidth", None)

df8

,Course Code,Course Name,EOS,Mid,Institution
1,AL801,Software Design with Virtual Reality and Gaming,304,328,Athlone Institute of Technology
2,AL802,Software Design with Cloud Computing,301,306,Athlone Institute of Technology
3,AL803,Software Design with Mobile Apps and Connected...,309,337,Athlone Institute of Technology
4,AL805,Network Management and Cloud Infrastructure,329,442,Athlone Institute of Technology
5,AL810,Quantity Surveying,307,349,Athlone Institute of Technology
...,...,...,...,...,...
960,WD200,Arts (options),221,296,Waterford Institute of Technology
961,WD210,Software Systems Development,271,329,Waterford Institute of Technology
962,WD211,Creative Computing,275,322,Waterford Institute of Technology
963,WD212,Recreation and Sport Management,274,311,Waterford Institute of Technology


In [7]:
# Extract dictionary mapping two-letter college codes to institution names
# This will be useful for the 2021 data
college_code = dict(zip(df8['Course Code'].str[:2].unique(), df8['Institution'].unique()))

In [8]:
# Examine EOS values which contain non-numeric characters
df8[df8['EOS'].str.contains(r'[^0-9#*]') == True]

,Course Code,Course Name,EOS,Mid,Institution
431,AD101,First Year Art & Design (Common Entry),# +mat,ic 550,National College of Art and Design
432,AD102,Graphic Design and Moving Image Design,# +mat,ic 635,National College of Art and Design
433,AD103,Textile & Surface Design and Jewellery & Objects,# +mat,ic 545,National College of Art and Design
434,AD202,Education & Design or Fine Art (Second Level T...,# +mat,ic 580,National College of Art and Design
435,AD204,Fine Art,# +mat,ic 600,National College of Art and Design
436,AD211,Fashion Design,# +mat,ic 600,National College of Art and Design
437,AD212,Product Design,# +mat,ic 600,National College of Art and Design
439,AD217,Illustration,# +mat,ic 625,National College of Art and Design
440,AD222,Interaction Design,# +mat,ic 657,National College of Art and Design
815,LM124,Mathematics (Math Science/Maths and Physics/Ec...,e)427,521,University of Limerick


In [9]:
# Examine Mid values which contain non-numeric characters
df8[df8['Mid'].str.contains(r'[^0-9]') == True]

,Course Code,Course Name,EOS,Mid,Institution
431,AD101,First Year Art & Design (Common Entry),# +mat,ic 550,National College of Art and Design
432,AD102,Graphic Design and Moving Image Design,# +mat,ic 635,National College of Art and Design
433,AD103,Textile & Surface Design and Jewellery & Objects,# +mat,ic 545,National College of Art and Design
434,AD202,Education & Design or Fine Art (Second Level T...,# +mat,ic 580,National College of Art and Design
435,AD204,Fine Art,# +mat,ic 600,National College of Art and Design
436,AD211,Fashion Design,# +mat,ic 600,National College of Art and Design
437,AD212,Product Design,# +mat,ic 600,National College of Art and Design
439,AD217,Illustration,# +mat,ic 625,National College of Art and Design
440,AD222,Interaction Design,# +mat,ic 657,National College of Art and Design


In [10]:
# Create new column indicating whether the course requires a test, interview or portfolio
# This is indicated by a '#' in the EOS column
df8['Test'] = df8['EOS'].str.contains('#', na=False)

# Create a column indicating courses where not all applicants at EOS point score were offered a place
# This is indicated by a '*' in the EOS column
df8['Not All'] = df8['EOS'].str.contains('\*', na=False)

# Create a column indicating courses where a matric is required
# This is indicated by the string '+matric' in the EOS column.
# However, the tabula table parsing has interpreted the r in matric as a cell boundary so only 'mat' 
# remains in the EOS column and 'ic' appears in the Mid column. The 'ic' will be dealt with next 
df8['Matric'] = df8['EOS'].str.contains('mat', na=False)


In [11]:
# Remove Non-digits from EOS and Mid columns and convert columns to numeric values, with NaNs where values are missing (errors = 'coerce')
# (Because NaN is a float, the whole columns must be floats)
df8['EOS'] = pd.to_numeric(df8['EOS'].str.replace(r'[^0-9]+', '', regex=True), errors='coerce')
df8['Mid'] = pd.to_numeric(df8['Mid'].str.replace(r'[^0-9]+', '', ), errors='coerce')

/tmp/ipykernel_79465/78865963.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df8['Mid'] = pd.to_numeric(df8['Mid'].str.replace(r'[^0-9]+', '', ), errors='coerce')


In [12]:
# Repair LM124 Course Name
df8.loc[df8['Course Code']=='LM124', 'Course Name'] += 'ce)'

#### Level 6 and 7

In [14]:
# read the entire pdf, extracting tables into a single dataframe
df67 = read_pdf(os.path.join(data_dir, "lvl76_19.pdf"), pages="all", multiple_tables=False)[0]
df67.head(10)

,Unnamed: 0,ADMISSION DATA 2019,Unnamed: 2,Unnamed: 3
0,NaN,End of Season,NaN,NaN
1,NaN,"Level 6, 7",NaN,NaN
2,NaN,The details given are for general information...,NaN,NaN
3,*,Not all on this points score were offered places,NaN,NaN
4,#,Test / Interview / Portfolio / Audition,NaN,NaN
5,AQA,All qualified applicants,NaN,NaN
6,NaN,NaN,NaN,NaN
7,Course Code,INSTITUTION and COURSE,EOS,Mid
8,NaN,Athlone Institute of Technology,NaN,NaN
9,AL600,Software Design,205,306


In [15]:
# Some text has been included in the dataframe. The actual table starts at row 7 with the row names
# Rename rows using row 7
df67.columns = df67.iloc[7]
df67.rename_axis(None, axis=1, inplace=True)


In [16]:
# Delete rows up to row 7
df67.drop(df67.index[range(0, 8)], axis=0, inplace=True)
df67.head(10)

,Course Code,INSTITUTION and COURSE,EOS,Mid
8,NaN,Athlone Institute of Technology,NaN,NaN
9,AL600,Software Design,205,306
10,AL601,Computer Engineering,196,272
11,AL602,Mechanical Engineering,258,424
12,AL604,Civil Engineering,252,360
13,AL630,Pharmacy Technician,306,366
14,AL631,Dental Nursing,326,379
15,AL632,Applied Science,243,372
16,AL650,Business,210,317
17,AL651,Music and Instrument Technology,AQA,296


In [17]:
df67[df67['Mid'].str.contains(r'[^0-9]') == True]
df67[df67['EOS'].str.contains(r'[^0-9]') == True]

,Course Code,INSTITUTION and COURSE,EOS,Mid
17,AL651,Music and Instrument Technology,AQA,296
18,AL660,Culinary Arts,AQA,216
19,AL661,Bar Supervision,AQA,216
20,AL663,Business (Sport and Recreation),AQA,270
36,AL763,Graphic Design (portfolio),#626,820
45,CW057,"Art (portfolio, Wexford)",#700,700
67,CW807,Sport Coaching and Business Management - GAA (...,#700,700
68,CW817,Sport Coaching and Business Management - Rugby...,#700,700
69,CW827,Sport Coaching and Business Management - Socce...,#700,700
78,PC404,Applied Social Studies - Professional Social Care,AQA,245


In [18]:
# Now this datframe can be cleaned up in the same manner as level 8
def cleanup(df):
    
    df = df.copy(deep=True)
    
    # Create a new column in the dataframe for institution name 
    # identify institution name rows as those containing null course codes
    # and add those institution names to the new institution column
    df['Institution'] = df[df['Course Code'].isnull()]['INSTITUTION and COURSE']
    df.rename(columns={'INSTITUTION and COURSE':'Course Name'}, inplace=True)

    # Fill empty fields in the institution column with the most recent non-na field
    df['Institution'] = df['Institution'].fillna(method='ffill')

    # Finally, remove rows containing only institution names
    df = df[df['Course Code'].notna()]

    # Create new column indicating whether the course requires a test, interview or portfolio
    # This is indicated by a '#' in the EOS column
    df['Test'] = df['EOS'].str.contains('#', na=False)

    # Create a column indicating courses where not all applicants at EOS point score were offered a place
    # This is indicated by a '*' in the EOS column
    df['Not All'] = df['EOS'].str.contains('\*', na=False)

    # Create a column indicating courses where a matric is required
    # This is indicated by the string '+matric' in the EOS column.
    # However, the tabula table parsing has interpreted the r in matric as a cell boundary so only 'mat' 
    # remains in the EOS column and 'ic' appears in the Mid column. The 'ic' will be dealt with next 
    df['Matric'] = df['EOS'].str.contains('mat', na=False)

    # Level 6 & 7 has a new code in EOS -- 'AQA' meaning All Qualified Applicants were offered a place
    # Create a new column for AQA
    df['AQA'] = df['EOS'].str.contains('AQA', na=False)

    # Remove Non-digits from EOS and Mid columns and convert columns to numeric values, with NaNs where values are missing (errors = 'coerce')
    # (Because NaN is a float, the whole columns must be floats)
    df['EOS'] = pd.to_numeric(df['EOS'].str.replace(r'[^0-9]+', '', regex=True), errors='coerce')
    df['Mid'] = pd.to_numeric(df['Mid'].str.replace(r'[^0-9]+', '', regex=True), errors='coerce')

    return df

df67 = cleanup(df67)        

In [19]:
df67.head(10)

,Course Code,Course Name,EOS,Mid,Institution,Test,Not All,Matric,AQA
9,AL600,Software Design,205.0,306.0,Athlone Institute of Technology,False,False,False,False
10,AL601,Computer Engineering,196.0,272.0,Athlone Institute of Technology,False,False,False,False
11,AL602,Mechanical Engineering,258.0,424.0,Athlone Institute of Technology,False,False,False,False
12,AL604,Civil Engineering,252.0,360.0,Athlone Institute of Technology,False,False,False,False
13,AL630,Pharmacy Technician,306.0,366.0,Athlone Institute of Technology,False,False,False,False
14,AL631,Dental Nursing,326.0,379.0,Athlone Institute of Technology,False,False,False,False
15,AL632,Applied Science,243.0,372.0,Athlone Institute of Technology,False,False,False,False
16,AL650,Business,210.0,317.0,Athlone Institute of Technology,False,False,False,False
17,AL651,Music and Instrument Technology,NaN,296.0,Athlone Institute of Technology,False,False,False,True
18,AL660,Culinary Arts,NaN,216.0,Athlone Institute of Technology,False,False,False,True


In [20]:
# Repair WD177 Course Name
df67.loc[df67['Course Code']=='WD177', 'Course Name'] += 'macy.)'

#### Merge dataframes

In [21]:
# add AQA column to level 8 dataframe
df8['AQA'] = False

# add level 8 column to both dataframes
df8['Level 8'] = True
df67['Level 8'] = False



In [22]:
# conatenate level with levels 6 & 7
df = pd.concat([df8, df67], ignore_index=True)

# Rename column names to include year   
df = df.rename({'Course Name': 'Course Name 2019', 
                'EOS': 'EOS 2019', 
                'Mid':'Mid 2019', 
                'Test':'Test 2019', 
                'Not All': 'NotAll 2019',
                'Matric': 'Matric 2019',
                'AQA': 'AQA 2019',
                'Level 8': 'Level8 2019'}, axis=1)

In [23]:
df.head()

,Course Code,Course Name 2019,EOS 2019,Mid 2019,Institution,Test 2019,NotAll 2019,Matric 2019,AQA 2019,Level8 2019
0,AL801,Software Design with Virtual Reality and Gaming,304.0,328.0,Athlone Institute of Technology,False,False,False,False,True
1,AL802,Software Design with Cloud Computing,301.0,306.0,Athlone Institute of Technology,False,False,False,False,True
2,AL803,Software Design with Mobile Apps and Connected...,309.0,337.0,Athlone Institute of Technology,False,False,False,False,True
3,AL805,Network Management and Cloud Infrastructure,329.0,442.0,Athlone Institute of Technology,False,False,False,False,True
4,AL810,Quantity Surveying,307.0,349.0,Athlone Institute of Technology,False,False,False,False,True


In [24]:
# export dataframe to csv
df.to_csv('data/cao/cao_points_2019.csv')

### 2020 Points Data

In [27]:
df2020 = pd.read_excel(os.path.join(data_dir, 'all2020.xlsx'), header=10)
df2020.columns
df2020.head()

,CATEGORY (i.e.ISCED description),COURSE TITLE,COURSE CODE2,R1 POINTS,R1 Random *,R2 POINTS,R2 Random*,EOS,EOS Random *,EOS Mid-point,...,avp,v,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
0,Business and administration,International Business,AC120,209,NaN,NaN,NaN,209,NaN,280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Humanities (except languages),Liberal Arts,AC137,252,NaN,NaN,NaN,252,NaN,270,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arts,"First Year Art & Design (Common Entry,portfolio)",AD101,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arts,Graphic Design and Moving Image Design (portfo...,AD102,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arts,Textile & Surface Design and Jewellery & Objec...,AD103,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 2021 Points Data

In [29]:
from bs4 import BeautifulSoup
from bs4 import SoupStrainer
import re

l8 = os.path.join(data_dir, 'lvl8_21.html')

# encoding=cp1252 necessary to decode some characters on page
with open(l8, 'r', encoding='cp1252') as f:

    # Soupstrainer allows us to specify what tags to include.
    # All the content we need is between <pre> tags
    pre = SoupStrainer('pre')
    soup = BeautifulSoup(f, 'html.parser', parse_only=pre)

# Section headers are all between bold tags so remove them
for match in soup.find_all('b'):
    match.decompose()





In [30]:
# Split remaining text into lines
lines = soup.text.split("\n")
# Regular expression to capture fields from lines
# Lines consist of 2 letters and 3 numbers, comprising the course code; some whitespace; 
# 50 characters which start with a non-whitespace character; 
fields = re.compile('^([A-Z]{2}[0-9]{3})\s+(\S.{49})\s+(\S+)\s+(\S+)?')
# column names
columns = ['Course Code', 'Course Name', 'Rnd1', 'Rnd2']

# Iterate through lines extracting fields from each line that contains data
data = []
for line in lines:
    if re.match(r'^[A-Z]', line):
        m = re.search(fields, line)
        if m:
            data.append(list(m.groups()))

# Create dataframe from extracted data
df = pd.DataFrame.from_records(data, columns=columns)
df.head()



,Course Code,Course Name,Rnd1,Rnd2
0,AL801,Software Design for Virtual Reality and Gaming...,300,None
1,AL802,Software Design in Artificial Intelligence for...,313,None
2,AL803,Software Design for Mobile Apps and Connected ...,350,None
3,AL805,Computer Engineering for Network Infrastructur...,321,None
4,AL810,Quantity Surveying ...,328,None


In [31]:
# Cleaning up
# Examine Mid values which contain non-numeric characters
df[df['Rnd2'].str.contains(r'[^0-9]') == True]

,Course Code,Course Name,Rnd1,Rnd2
14,AL841,Athletic and Rehabilitation Therapy ...,477,476*
29,AL870,Applied Psychology ...,484*,467*
74,CR121,Music at CIT Cork School of Music ...,#904,#904
108,CR700,Theatre and Drama Studies at CIT Cork School o...,#609,#609
113,CK105,Film and Screen Media - 3 years or 4 years (In...,421,420*
...,...,...,...,...
654,GY501,Medicine (five year/six year course HPAT requ...,#737*,#737*
767,LM090,Physical Education with concurrent Teacher Edu...,552,544*
826,MH801,Early Childhood - Teaching and Learning (part-...,AQA,AQA
828,AS002,Education Home Economics and Religious Ed - w...,464,452*


In [32]:
# Create new columns to hold information currently designated by *'s and #'s in numeric columns
# Some cut and paste from 2019 above 
# TODO: Clean up repetitive code

# Create new column indicating whether the course requires a test, interview or portfolio
# This is indicated by a '#' in the Rnd1 or Rnd2 column
df['Test'] = df['Rnd1'].str.contains('#', na=False) | df['Rnd2'].str.contains('#', na=False)

# Create a column indicating courses where not all applicants at Rnd1 point score were offered a place
# This is indicated by a '*' in the Rnd1 or Rnd2 column
df['Not All'] = df['Rnd1'].str.contains('\*', na=False) | df['Rnd2'].str.contains('#', na=False)

# Create a new column for AQA meaning All Qualified Applicants were offered a place
df['AQA'] = df['Rnd1'].str.contains('AQA', na=False) | df['Rnd1'].str.contains('AQA', na=False)

# Remove Non-digits from EOS and Mid columns and convert columns to numeric values, with NaNs where values are missing (errors = 'coerce')
# (Because NaN is a float, the whole columns must be floats)
df['Rnd1'] = pd.to_numeric(df['Rnd1'].str.replace(r'[^0-9]+', '', regex=True), errors='coerce')
df['Rnd2'] = pd.to_numeric(df['Rnd2'].str.replace(r'[^0-9]+', '', regex=True), errors='coerce')

In [36]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(df)

,Course Code,Course Name,Rnd1,Rnd2,Test,Not All,AQA
0,AL801,Software Design for Virtual Reality and Gaming...,300.0,NaN,False,False,False
1,AL802,Software Design in Artificial Intelligence for...,313.0,NaN,False,False,False
2,AL803,Software Design for Mobile Apps and Connected ...,350.0,NaN,False,False,False
3,AL805,Computer Engineering for Network Infrastructur...,321.0,NaN,False,False,False
4,AL810,Quantity Surveying ...,328.0,NaN,False,False,False
5,AL820,Mechanical and Polymer Engineering ...,327.0,NaN,False,False,False
6,AL830,General Nursing ...,451.0,444.0,False,True,False
7,AL832,Mental Health Nursing ...,440.0,431.0,False,True,False
8,AL835,Pharmacology ...,356.0,NaN,False,False,False
9,AL836,Nutrition and Health Science ...,346.0,NaN,False,False,False


## Analysing the data

In [ ]:
df[['EOS', 'Mid']].hist(bins=50, figsize=(20, 5))

## Conclusion

## References

[1] https://www.independent.ie/life/family/learning/understanding-your-cao-course-guide-26505318.html